In [1]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession, Row
import pandas as pd


In [2]:
spark = SparkSession.builder.appName("rdd-app").config("spark.config.option", "value").getOrCreate()
scfg = SparkConf().setAppName('rdd-app')
sc = spark.sparkContext

In [3]:
import string

text_file = '/user/student/shakespeare/tragedy/hamlet.txt'
text = sc.textFile(text_file)

In [4]:
def strip_punc(s):
    return s.translate(str.maketrans('', '', string.punctuation)).split(' ')

def search_word_in_line(word):
    count = 1
    for line in text.collect():
        if word in strip_punc(line):
            print('{}. {}'.format(count, line))
        count += 1

In [5]:
flatmap = text.flatMap(lambda line: line.translate(str.maketrans('', '', string.punctuation)).split(' '))
map = flatmap.map(lambda word: (word, 1))
reduced = map.reduceByKey(lambda a, b: a + b)

In [6]:
counts = text.flatMap(lambda line: line.translate(str.maketrans('', '', string.punctuation)).split(' '))\
             .map(lambda word: (word, 1))\
             .reduceByKey(lambda a, b: a + b)    

In [7]:
word = "purpose"
for count in reduced.collect():
    # kv = str(count).translate(str.maketrans('', '', string.punctuation)).split(' ')
    kv = strip_punc(str(count))
    if word == kv[0]:
        print('Found \'{}\' occurs \'{}\' times'.format(kv[0], kv[1])) 
        search_word_in_line(word)
        break

Found 'purpose' occurs '11' times
2599.     Why, any thing, but to the purpose. You were sent
2926.     Black as his purpose, did the night resemble
3216.     And drive his purpose on to these delights.
3540.     from the purpose of playing, whose end, both at the
3909.     The passion ending, doth the purpose lose.
4766.     Is but to whet thy almost blunted purpose.
6202.     And, for that purpose, I'll anoint my sword.
6227.     Our purpose may hold there.
6378.     purpose, confess thyself--
7328.     king hold his purpose, I will win for him an I can;
7376.     I am constant to my purpose; they follow the king's


## Manipulating airline performance data

In [8]:

from pyspark.sql.types import Row
from datetime import datetime


In [9]:
data_by_year = '/user/student/airline/2001.csv'
airline_performance = spark.read.option("header", "true").csv(data_by_year)

In [10]:
airline_performance.head()

Row(Year='2001', Month='1', DayofMonth='17', DayOfWeek='3', DepTime='1806', CRSDepTime='1810', ArrTime='1931', CRSArrTime='1934', UniqueCarrier='US', FlightNum='375', TailNum='N700��', ActualElapsedTime='85', CRSElapsedTime='84', AirTime='60', ArrDelay='-3', DepDelay='-4', Origin='BWI', Dest='CLT', Distance='361', TaxiIn='5', TaxiOut='20', Cancelled='0', CancellationCode='NA', Diverted='0', CarrierDelay='NA', WeatherDelay='NA', NASDelay='NA', SecurityDelay='NA', LateAircraftDelay='NA')

In [11]:
airline_performance

DataFrame[Year: string, Month: string, DayofMonth: string, DayOfWeek: string, DepTime: string, CRSDepTime: string, ArrTime: string, CRSArrTime: string, UniqueCarrier: string, FlightNum: string, TailNum: string, ActualElapsedTime: string, CRSElapsedTime: string, AirTime: string, ArrDelay: string, DepDelay: string, Origin: string, Dest: string, Distance: string, TaxiIn: string, TaxiOut: string, Cancelled: string, CancellationCode: string, Diverted: string, CarrierDelay: string, WeatherDelay: string, NASDelay: string, SecurityDelay: string, LateAircraftDelay: string]

In [12]:
from pyspark.sql.types import IntegerType
airline_performance = airline_performance.withColumn("ArrDelay", airline_performance["ArrDelay"].cast(IntegerType()))
airline_performance = airline_performance.withColumn("DepDelay", airline_performance["DepDelay"].cast(IntegerType()))

In [13]:
airline_performance.describe(['ArrDelay']).show()
airline_performance.describe(['DepDelay']).show()

+-------+------------------+
|summary|          ArrDelay|
+-------+------------------+
|  count|           5723673|
|   mean| 5.528248731190619|
| stddev|31.429291168399615|
|    min|             -1116|
|    max|              1688|
+-------+------------------+

+-------+------------------+
|summary|          DepDelay|
+-------+------------------+
|  count|           5736582|
|   mean| 8.154837497311117|
| stddev|28.348469290963077|
|    min|              -204|
|    max|              1692|
+-------+------------------+



In [14]:
import pyspark.sql.functions as F
airline_performance.select(airline_performance.Dest,F.when(airline_performance.ArrDelay > 0, 1).otherwise(0)).show()

+----+------------------------------------------+
|Dest|CASE WHEN (ArrDelay > 0) THEN 1 ELSE 0 END|
+----+------------------------------------------+
| CLT|                                         0|
| CLT|                                         1|
| CLT|                                         1|
| CLT|                                         1|
| CLT|                                         1|
| CLT|                                         0|
| CLT|                                         0|
| CLT|                                         0|
| CLT|                                         0|
| CLT|                                         0|
| CLT|                                         0|
| CLT|                                         0|
| CLT|                                         0|
| CLT|                                         0|
| CLT|                                         0|
| MHT|                                         1|
| MHT|                                         1|


In [15]:
airline_performance = airline_performance.withColumn('ArrDelayCount',F.when(airline_performance.ArrDelay > 0, 1).otherwise(0))
airline_performance = airline_performance.withColumn('DepDelayCount',F.when(airline_performance.DepDelay > 0, 1).otherwise(0))
airline_performance.show(2)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+-------------+-------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|ArrDelayCount|DepDelayCount|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+-------------+-------------+
|2001|    

In [16]:
#Most Depature Delay Airport 2001
airline_performance\
.filter(F.col('DepDelayCount') == 1)\
.groupBy('Year','Origin')\
.count()\
.orderBy('count',ascending=False)\
.show(1)

+----+------+------+
|Year|Origin| count|
+----+------+------+
|2001|   ORD|154334|
+----+------+------+
only showing top 1 row



In [17]:
#Least Depature Delay Airport 2001
airline_performance\
.filter(F.col('DepDelayCount') == 1)\
.groupBy('Year','Origin')\
.count()\
.orderBy('count',ascending=True)\
.show(1)

+----+------+-----+
|Year|Origin|count|
+----+------+-----+
|2001|   ACY|    1|
+----+------+-----+
only showing top 1 row



In [18]:
#Most Arrival Delay Airport 2001
airline_performance\
.filter(F.col('ArrDelayCount') == 1)\
.groupBy('Year','Dest')\
.count()\
.orderBy('count',ascending=False)\
.show(1)

+----+----+------+
|Year|Dest| count|
+----+----+------+
|2001| ORD|136932|
+----+----+------+
only showing top 1 row



In [19]:
#least Arrival Delay Airport 2001
airline_performance\
.filter(F.col('ArrDelayCount') == 1)\
.groupBy('Year','Dest')\
.count()\
.orderBy('count',ascending=True)\
.show(1)

+----+----+-----+
|Year|Dest|count|
+----+----+-----+
|2001| RDR|   24|
+----+----+-----+
only showing top 1 row



In [20]:
#Most Arrival Delay Flight 2001
airline_performance\
.filter(F.col('ArrDelayCount') == 1)\
.groupBy('Year','UniqueCarrier','FlightNum')\
.count()\
.orderBy('count',ascending=False)\
.show(1)

+----+-------------+---------+-----+
|Year|UniqueCarrier|FlightNum|count|
+----+-------------+---------+-----+
|2001|           AS|       64|  992|
+----+-------------+---------+-----+
only showing top 1 row



In [21]:
#Least Arrival Delay Flight 2001
airline_performance\
.filter(F.col('ArrDelayCount') == 1)\
.groupBy('Year','UniqueCarrier','FlightNum')\
.count()\
.orderBy('count',ascending=True)\
.show(1)

+----+-------------+---------+-----+
|Year|UniqueCarrier|FlightNum|count|
+----+-------------+---------+-----+
|2001|           HP|     2227|    1|
+----+-------------+---------+-----+
only showing top 1 row



In [22]:
#Most Departure Delay Flight 2001
airline_performance\
.filter(F.col('DepDelayCount') == 1)\
.groupBy('Year','UniqueCarrier','FlightNum')\
.count()\
.orderBy('count',ascending=False)\
.show(1)

+----+-------------+---------+-----+
|Year|UniqueCarrier|FlightNum|count|
+----+-------------+---------+-----+
|2001|           WN|     1575|  927|
+----+-------------+---------+-----+
only showing top 1 row



In [23]:
#Least Departure Delay Flight 2001
airline_performance\
.filter(F.col('DepDelayCount') == 1)\
.groupBy('Year','UniqueCarrier','FlightNum')\
.count()\
.orderBy('count',ascending=True)\
.show(1)

+----+-------------+---------+-----+
|Year|UniqueCarrier|FlightNum|count|
+----+-------------+---------+-----+
|2001|           US|     2110|    1|
+----+-------------+---------+-----+
only showing top 1 row



In [24]:
airline_performance\
.groupBy('Year','UniqueCarrier')\
.agg(F.mean('ArrDelay'))\
.show()

+----+-------------+------------------+
|Year|UniqueCarrier|     avg(ArrDelay)|
+----+-------------+------------------+
|2001|           CO|3.3232846269845395|
|2001|           AA| 5.782605193751565|
|2001|           HP| 8.324058042302017|
|2001|           AQ| 4.888563890800643|
|2001|           US| 3.513551631755064|
|2001|           WN| 4.855821933876753|
|2001|           TW|4.2484949608438285|
|2001|           UA| 8.625530044317387|
|2001|           MQ| 7.761099295787077|
|2001|           AS| 11.71687910390112|
|2001|           NW| 3.151802560581363|
|2001|           DL| 4.984308254185548|
+----+-------------+------------------+



In [25]:
airline_performance\
.groupBy('Year','UniqueCarrier')\
.agg(F.mean('DepDelay'))\
.show()

+----+-------------+------------------+
|Year|UniqueCarrier|     avg(DepDelay)|
+----+-------------+------------------+
|2001|           CO| 5.541168762029567|
|2001|           AA| 9.026835875605144|
|2001|           HP| 9.858909626719058|
|2001|           AQ| 4.199717065896882|
|2001|           US| 5.881444755763281|
|2001|           WN|  8.76108650459363|
|2001|           TW| 6.013394762610158|
|2001|           UA| 11.03689434786824|
|2001|           MQ|10.448874098089139|
|2001|           AS|10.971318930833478|
|2001|           NW|5.7991100783433644|
|2001|           DL| 7.500545855609975|
+----+-------------+------------------+



----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 44804)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/home/student/dev/spark/python/pyspark/accumulators.py", line 268, in handle
    poll(accum_updates)
  File "/home/student/dev/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/home/student/dev/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/home/student/dev/spark/python/